Importações

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression 
from app import func as f

path_arquivo="" #caminho do arquivo a ser consumido, incluindo o nome do mesmo

df=pd.read_csv(path_arquivo, encoding="utf-16", header=0, delim_whitespace=True, index_col=0)

df

Regressão Linear para obter taxa de aquecimento


In [ ]:
model = LinearRegression()
x=np.array(df["Time(s)"]).reshape(-1,1)
y=np.array(df["Furnace_Temperature(°C)"])

model.fit(x,y)
m=model.coef_[0]
b=model.intercept_

with open("Outputs/Dados Coletados - Sulfato de Cobre.txt", "w", encoding='utf-8') as arquivo:
    arquivo.write("Taxa de aquecimento: %f\nCoeficiente Linear: %f\n" %(m,b))

df["Derivada(Txt)"]=f.derivada(df["Time(s)"],df["Furnace_Temperature(°C)"])


Plot DTA e DTG

In [ ]:
massaportemperatura=f.derivada(df["Sample_Temperature(°C)"],df["TG(%)"])
df["Derivada(Mass,temp)"]=massaportemperatura

fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('Temperatura da Amostra (°C)')
ax1.set_ylabel("Massa da Amostra (%)", color=color)
ax1.plot(df["Sample_Temperature(°C)"],df["TG(%)"], label="Dados do Aquecimento",color=color)
plt.title("Variação da massa com a temperatura")

ax1.tick_params(axis='y', labelcolor = color)

ax2=ax1.twinx()

color = 'tab:blue'
ax2.set_ylabel('Derivada (%/°C)', color=color)

#limite inferior adicionado para eliminar o ruído no início do gráfico
ax2.plot(df[df["Time(s)"]>115]["Sample_Temperature(°C)"],massaportemperatura[116:],color=color)
ax2.tick_params(axis='y', labelcolor = color)

fig.tight_layout()

fig=plt.gcf()

plt.show()

fig.savefig("Outputs/Variação da massa com a mudança da temperatura.png")
plt.close(fig)


Minimos e Máximos locais

In [ ]:
#Com o gráfico em mão, é possível ver a localização aproximada dos picos. Esse trecho do código é específico para cada caso de estudo.
#O procedimento "geral" é colocar o tipo de pico ("max" ou "min") e o intervalo onde ele se encontra (visível no gráfico).

minimo_2=f.picolocal(df["Derivada(Mass,temp)"],"min",351,900)

#indice para limitar a busca pelo primeiro maximo
indice_inflex_1=df.index[df["Derivada(Mass,temp)"]==minimo_2].values[0]

maximo_1=f.picolocal(df["Derivada(Mass,temp)"],"max",600,indice_inflex_1-100)

#indice para limitar a busca pelo primeiro minimo
indice_max=df.index[df["Derivada(Mass,temp)"]==maximo_1].values[0]

minimo_1=f.picolocal(df["Derivada(Mass,temp)"],"min",600,indice_max)


minimo_3=f.picolocal(df["Derivada(Mass,temp)"],"min",984,1500)

minimo_5=f.picolocal(df["Derivada(Mass,temp)"],"min",1100,5000)

maximo_2=f.picolocal(df["Derivada(Mass,temp)"],"max",4400,4700)


minimo_4=f.picolocal(df["Derivada(Mass,temp)"],"min",4200,4415)


print(df[df["Derivada(Mass,temp)"]==minimo_1]["Sample_Temperature(°C)"])
print(df[df["Derivada(Mass,temp)"]==maximo_1]["Sample_Temperature(°C)"])
print(df[df["Derivada(Mass,temp)"]==minimo_2]["Sample_Temperature(°C)"])
print(df[df["Derivada(Mass,temp)"]==minimo_3]["Sample_Temperature(°C)"])
print(df[df["Derivada(Mass,temp)"]==minimo_4]["Sample_Temperature(°C)"])
print(df[df["Derivada(Mass,temp)"]==maximo_2]["Sample_Temperature(°C)"])
print(df[df["Derivada(Mass,temp)"]==minimo_5]["Sample_Temperature(°C)"])

Encontrando máximo e mínimo da derivada (pontos de inflexão) e reta tangente

In [ ]:
#ponto de inflexão minimo 1
T_inf1=df[df["Derivada(Mass,temp)"]==minimo_1]["Sample_Temperature(°C)"].values[0]
F_inf1=df[df["Derivada(Mass,temp)"]==minimo_1]["TG(%)"].values[0]

#ponto de inflexão minimo 2
T_inf2=df[df["Derivada(Mass,temp)"]==minimo_2]["Sample_Temperature(°C)"].values[0]
F_inf2=df[df["Derivada(Mass,temp)"]==minimo_2]["TG(%)"].values[0]

#ponto de inflexão minimo 3

T_inf3=df[df["Derivada(Mass,temp)"]==minimo_3]["Sample_Temperature(°C)"].values[0]
F_inf3=df[df["Derivada(Mass,temp)"]==minimo_3]["TG(%)"].values[0]

#ponto de inflexão minimo 4
T_inf4=df[df["Derivada(Mass,temp)"]==minimo_4]["Sample_Temperature(°C)"].values[0]
F_inf4=df[df["Derivada(Mass,temp)"]==minimo_4]["TG(%)"].values[0]

#ponto de inflexão minimo 5
T_inf5=df[df["Derivada(Mass,temp)"]==minimo_5]["Sample_Temperature(°C)"].values[0]
F_inf5=df[df["Derivada(Mass,temp)"]==minimo_5]["TG(%)"].values[0]


c_1=F_inf1-(minimo_1*T_inf1) #coeficiente linear da reta tangente 1
c_2=F_inf2-(minimo_2*T_inf2) #coeficiente linear da reta tangente 2
c_3=F_inf3-(minimo_3*T_inf3) #coeficiente linear da reta tangente 3
c_4=F_inf4-(minimo_4*T_inf4) #coeficiente linear da reta tangente 4
c_5=F_inf5-(minimo_5*T_inf5) #coeficiente linear da reta tangente 5

Perda de massa em cada evento

In [ ]:
#Essa parte também depende de cada caso. O procedimento "geral" é a partir das inflexões encontradas para cada evento, determinar as linhas de base (onset e endset).

#-----------------------------------

#----------------------------------- Procedimento: encontre o indice da inflexão, capture a derivada e seu indice
indice_inflex_1=df.index[df["Derivada(Mass,temp)"]==minimo_1]                                                       #indice do ponto de inflexão para limitar o intervalo de busca
T_base_set_1=f.linhabaseonset(df["Derivada(Mass,temp)"],df["Sample_Temperature(°C)"],indice_inflex_1)[0]            #Pelo indice, encontra ao redor a ultima temperatura na linha de base
indice_base_onset_1=f.linhabaseonset(df["Derivada(Mass,temp)"],df["Sample_Temperature(°C)"],indice_inflex_1)[1]     #e seu valor de fluxo

#Tendset e seu indice para encontrar o Fluxo de Endset
T_base_endset_1=f.linhabaseendset(df["Derivada(Mass,temp)"],df["Sample_Temperature(°C)"],indice_inflex_1)[0]
indice_base_endset_1=f.linhabaseendset(df["Derivada(Mass,temp)"],df["Sample_Temperature(°C)"],indice_inflex_1)[1]

evento1_massa_inicial=df.loc[indice_base_onset_1]["TG(%)"].values[0]
evento1_massa_final=df.loc[indice_base_endset_1]["TG(%)"].values[0]

perda1=evento1_massa_inicial - evento1_massa_final

#A perda de massa de cada evento é dada pela diferença entre a massa inicial (massa no onset) e a massa final (massa no endset).
#-----------------------------------

#-----------------------------------

indice_inflex_2=df.index[df["Derivada(Mass,temp)"]==minimo_3]
T_base_set_2=f.linhabaseonset(df["Derivada(Mass,temp)"],df["Sample_Temperature(°C)"],indice_inflex_2)[0]
indice_base_onset_2=f.linhabaseonset(df["Derivada(Mass,temp)"],df["Sample_Temperature(°C)"],indice_inflex_2)[1]

#Tendset e seu indice para encontrar o Fluxo de Endset
T_base_endset_2=f.linhabaseendset(df["Derivada(Mass,temp)"],df["Sample_Temperature(°C)"],indice_inflex_2)[0]
indice_base_endset_2=f.linhabaseendset(df["Derivada(Mass,temp)"],df["Sample_Temperature(°C)"],indice_inflex_2)[1]

evento2_massa_inicial=df.loc[indice_base_onset_2]["TG(%)"].values[0]
evento2_massa_final=df.loc[indice_base_endset_2]["TG(%)"].values[0]

perda2=evento2_massa_inicial - evento2_massa_final

#-----------------------------------
#-----------------------------------

indice_inflex_3=df.index[df["Derivada(Mass,temp)"]==minimo_4]
T_base_set_3=f.linhabaseonset(df["Derivada(Mass,temp)"],df["Sample_Temperature(°C)"],indice_inflex_3)[0]
indice_base_onset_3=f.linhabaseonset(df["Derivada(Mass,temp)"],df["Sample_Temperature(°C)"],indice_inflex_3)[1]

#Tendset e seu indice para encontrar o Fluxo de Endset
T_base_endset_3=f.linhabaseendset(df["Derivada(Mass,temp)"],df["Sample_Temperature(°C)"],indice_inflex_3,-0.2)[0]
indice_base_endset_3=f.linhabaseendset(df["Derivada(Mass,temp)"],df["Sample_Temperature(°C)"],indice_inflex_3,-0.1)[1]

evento3_massa_inicial=df.loc[indice_base_onset_3]["TG(%)"].values[0]
evento3_massa_final=df.loc[indice_base_endset_3]["TG(%)"].values[0]

perda3=evento3_massa_inicial - evento3_massa_final

with open("Outputs/Dados Coletados - Sulfato de Cobre.txt", "a", encoding='utf-8') as arquivo:
    arquivo.write("Perda de de massa no evento 1: %f %%\nPerda de de massa no evento 2: %f %%\nPerda de de massa no evento 3: %f %%\n" %(perda1,perda2,perda3))

Tinicio, Tonset, Tendset, Tfinal e Tpico

In [ ]:
#As temperaturas de início (Tonset) e fim (Tendset) são justamente os pontos das linhas de base.
#Consideramos como temperatura de onset e endset os pontos de interseção entre as linhas de base e as retas tangentes nos pontos de inflexão

T_onset_ev1=(evento1_massa_inicial-c_1)/minimo_1
T_onset_ev2=(evento2_massa_inicial-c_3)/minimo_3
T_onset_ev3=(evento3_massa_inicial-c_4)/minimo_4

T_endset_ev1=(evento1_massa_final-c_2)/minimo_2
T_endset_ev2=(evento2_massa_final-c_3)/minimo_3
T_endset_ev3=(evento3_massa_final-c_5)/minimo_5

with open("Outputs/Dados Coletados - Sulfato de Cobre.txt", "a", encoding='utf-8') as arquivo:
    arquivo.write("*** Evento 1 ***\nTemperatura de início: %f °C\nTemperatura de onset: %f °C\nTemperatura de pico: %F °C\nTemperatura de endset: %f °C\nTemperatura final: %f °C\n\n" %(T_base_set_1.values[0],T_onset_ev1,T_inf2,T_endset_ev1,T_base_endset_1.values[0]))
    arquivo.write("*** Evento 2 ***\nTemperatura de início: %f °C\nTemperatura de onset: %f °C\nTemperatura de pico: %F °C\nTemperatura de endset: %f °C\nTemperatura final: %f °C\n\n" %(T_base_set_2.values[0],T_onset_ev2,T_inf3,T_endset_ev2,T_base_endset_2.values[0]))
    arquivo.write("*** Evento 3 ***\nTemperatura de início: %f °C\nTemperatura de onset: %f °C\nTemperatura de pico: %F °C\nTemperatura de endset: %f °C\nTemperatura final: %f °C\n" %(T_base_set_3.values[0],T_onset_ev3,T_inf5,T_endset_ev3,T_base_endset_3.values[0]))
